In [1]:
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sns
import os
from math import sqrt
from google.colab import files

In [2]:
uploaded = files.upload()

Saving covid_19_india.csv to covid_19_india.csv
Saving covid_vaccine_statewise.csv to covid_vaccine_statewise.csv
Saving StatewiseTestingDetails.csv to StatewiseTestingDetails.csv


In [218]:
Covid_Testing = pd.read_csv('StatewiseTestingDetails.csv')
Covid_Testing.head()

,Date,State,TotalSamples,Negative,Positive
0,2020-04-17,Andaman and Nicobar Islands,1403.0,1210,12.0
1,2020-04-24,Andaman and Nicobar Islands,2679.0,NaN,27.0
2,2020-04-27,Andaman and Nicobar Islands,2848.0,NaN,33.0
3,2020-05-01,Andaman and Nicobar Islands,3754.0,NaN,33.0
4,2020-05-16,Andaman and Nicobar Islands,6677.0,NaN,33.0


In [219]:
Covid_Summary = pd.read_csv('covid_19_india.csv')
Covid_Summary.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1.0,2020-01-30,6:00 PM,Kerala,1,0,0.0,0.0,1.0
1,2.0,2020-01-31,6:00 PM,Kerala,1,0,0.0,0.0,1.0
2,3.0,2020-02-01,6:00 PM,Kerala,2,0,0.0,0.0,2.0
3,4.0,2020-02-02,6:00 PM,Kerala,3,0,0.0,0.0,3.0
4,5.0,2020-02-03,6:00 PM,Kerala,3,0,0.0,0.0,3.0


In [220]:
Covid_Summary.isnull().sum()

Sno                         28
Date                        28
Time                        28
State/UnionTerritory        28
ConfirmedIndianNational     28
ConfirmedForeignNational    28
Cured                       28
Deaths                      28
Confirmed                   28
dtype: int64

In [221]:
Covid_Summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15114 entries, 0 to 15113
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Sno                       15086 non-null  float64
 1   Date                      15086 non-null  object 
 2   Time                      15086 non-null  object 
 3   State/UnionTerritory      15086 non-null  object 
 4   ConfirmedIndianNational   15086 non-null  object 
 5   ConfirmedForeignNational  15086 non-null  object 
 6   Cured                     15086 non-null  float64
 7   Deaths                    15086 non-null  float64
 8   Confirmed                 15086 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.0+ MB


In [222]:
Covid_Summary['Date'] = pd.to_datetime(Covid_Summary['Date'],format="%Y/%m/%d")
Covid_Testing['Date'] = pd.to_datetime(Covid_Testing['Date'],format="%Y/%m/%d")

In [223]:
Covid_Summary = Covid_Summary.drop(['Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
Covid_Summary.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-01-30,6:00 PM,Kerala,0.0,0.0,1.0
1,2020-01-31,6:00 PM,Kerala,0.0,0.0,1.0
2,2020-02-01,6:00 PM,Kerala,0.0,0.0,2.0
3,2020-02-02,6:00 PM,Kerala,0.0,0.0,3.0
4,2020-02-03,6:00 PM,Kerala,0.0,0.0,3.0


In [224]:
Covid_Summary['Active'] = Covid_Summary['Confirmed'] - Covid_Summary['Cured'] - Covid_Summary['Deaths']
Covid_Summary.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active
0,2020-01-30,6:00 PM,Kerala,0.0,0.0,1.0,1.0
1,2020-01-31,6:00 PM,Kerala,0.0,0.0,1.0,1.0
2,2020-02-01,6:00 PM,Kerala,0.0,0.0,2.0,2.0
3,2020-02-02,6:00 PM,Kerala,0.0,0.0,3.0,3.0
4,2020-02-03,6:00 PM,Kerala,0.0,0.0,3.0,3.0


In [225]:
india_cases = Covid_Summary[Covid_Summary['Date'] == Covid_Summary['Date'].max()].copy().fillna(0)
india_cases = india_cases.drop(['Time'], axis=1)

In [226]:
india_cases.head()

,Date,State/UnionTerritory,Cured,Deaths,Confirmed,Active
15050,2021-05-19,Andaman and Nicobar Islands,6359.0,92.0,6674.0,223.0
15051,2021-05-19,Andhra Pradesh,1254291.0,9580.0,1475372.0,211501.0
15052,2021-05-19,Arunachal Pradesh,19977.0,88.0,22462.0,2397.0
15053,2021-05-19,Assam,290774.0,2344.0,340858.0,47740.0
15054,2021-05-19,Bihar,595377.0,4039.0,664115.0,64699.0


In [227]:
Trend = Covid_Summary.groupby(['Date'])['Confirmed', 'Deaths','Cured',].sum().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [228]:
Trend.head()

,Date,Confirmed,Deaths,Cured
0,2020-01-30,1.0,0.0,0.0
1,2020-01-31,1.0,0.0,0.0
2,2020-02-01,2.0,0.0,0.0
3,2020-02-02,3.0,0.0,0.0
4,2020-02-03,3.0,0.0,0.0


In [229]:
vaccination = pd.read_csv("covid_vaccine_statewise.csv")

In [230]:
vaccination.head()

,Updated On,State,Total Individuals Vaccinated,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,AEFI,18-30 years (Age),30-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered
0,16/01/2021,India,48276.0,3455.0,2957.0,48276.0,0.0,23757.0,24517.0,2.0,579.0,47697.0,NaN,NaN,NaN,NaN,NaN,48276
1,17/01/2021,India,58604.0,8532.0,4954.0,58604.0,0.0,27348.0,31252.0,4.0,635.0,57969.0,NaN,NaN,NaN,NaN,NaN,58604
2,18/01/2021,India,99449.0,13611.0,6583.0,99449.0,0.0,41361.0,58083.0,5.0,1299.0,98150.0,NaN,NaN,NaN,NaN,NaN,99449
3,19/01/2021,India,195525.0,17855.0,7951.0,195525.0,0.0,81901.0,113613.0,11.0,3017.0,192508.0,NaN,NaN,NaN,NaN,NaN,195525
4,20/01/2021,India,251280.0,25472.0,10504.0,251280.0,0.0,98111.0,153145.0,24.0,3946.0,247334.0,NaN,NaN,NaN,NaN,NaN,251280


In [231]:
vaccination['Total Vaccinatons'] = vaccination['First Dose Administered']+vaccination['Second Dose Administered']

#Renaming columns
vaccination.rename(columns = {'Updated On':'Date'}, inplace = True)

In [232]:
vaccination.head()

,Date,State,Total Individuals Vaccinated,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,AEFI,18-30 years (Age),30-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered,Total Vaccinatons
0,16/01/2021,India,48276.0,3455.0,2957.0,48276.0,0.0,23757.0,24517.0,2.0,579.0,47697.0,NaN,NaN,NaN,NaN,NaN,48276,48276.0
1,17/01/2021,India,58604.0,8532.0,4954.0,58604.0,0.0,27348.0,31252.0,4.0,635.0,57969.0,NaN,NaN,NaN,NaN,NaN,58604,58604.0
2,18/01/2021,India,99449.0,13611.0,6583.0,99449.0,0.0,41361.0,58083.0,5.0,1299.0,98150.0,NaN,NaN,NaN,NaN,NaN,99449,99449.0
3,19/01/2021,India,195525.0,17855.0,7951.0,195525.0,0.0,81901.0,113613.0,11.0,3017.0,192508.0,NaN,NaN,NaN,NaN,NaN,195525,195525.0
4,20/01/2021,India,251280.0,25472.0,10504.0,251280.0,0.0,98111.0,153145.0,24.0,3946.0,247334.0,NaN,NaN,NaN,NaN,NaN,251280,251280.0


In [233]:
vaccination=vaccination.fillna(vaccination.median())

In [234]:
vaccination.head()

,Date,State,Total Individuals Vaccinated,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,AEFI,18-30 years (Age),30-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered,Total Vaccinatons
0,16/01/2021,India,48276.0,3455.0,2957.0,48276.0,0.0,23757.0,24517.0,2.0,579.0,47697.0,263.0,36091.5,191850.5,553008.5,631321.0,48276,48276.0
1,17/01/2021,India,58604.0,8532.0,4954.0,58604.0,0.0,27348.0,31252.0,4.0,635.0,57969.0,263.0,36091.5,191850.5,553008.5,631321.0,58604,58604.0
2,18/01/2021,India,99449.0,13611.0,6583.0,99449.0,0.0,41361.0,58083.0,5.0,1299.0,98150.0,263.0,36091.5,191850.5,553008.5,631321.0,99449,99449.0
3,19/01/2021,India,195525.0,17855.0,7951.0,195525.0,0.0,81901.0,113613.0,11.0,3017.0,192508.0,263.0,36091.5,191850.5,553008.5,631321.0,195525,195525.0
4,20/01/2021,India,251280.0,25472.0,10504.0,251280.0,0.0,98111.0,153145.0,24.0,3946.0,247334.0,263.0,36091.5,191850.5,553008.5,631321.0,251280,251280.0


In [235]:
vaccination.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4589 entries, 0 to 4588
Data columns (total 19 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Date                                 4589 non-null   object 
 1   State                                4589 non-null   object 
 2   Total Individuals Vaccinated         4589 non-null   float64
 3   Total Sessions Conducted             4589 non-null   float64
 4   Total Sites                          4589 non-null   float64
 5   First Dose Administered              4589 non-null   float64
 6   Second Dose Administered             4589 non-null   float64
 7   Male(Individuals Vaccinated)         4589 non-null   float64
 8   Female(Individuals Vaccinated)       4589 non-null   float64
 9   Transgender(Individuals Vaccinated)  4589 non-null   float64
 10  Total Covaxin Administered           4589 non-null   float64
 11  Total CoviShield Administered 

In [236]:
vaccination = vaccination.drop(['Total Sessions Conducted'], axis=1)

In [237]:
india_cases = india_cases.rename(columns={'State/UnionTerritory':'State'})

In [238]:
india_cases = india_cases.append(vaccination, sort=False)

In [239]:
india_cases.head()

,Date,State,Cured,Deaths,Confirmed,Active,Total Individuals Vaccinated,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,AEFI,18-30 years (Age),30-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered,Total Vaccinatons
15050,2021-05-19 00:00:00,Andaman and Nicobar Islands,6359.0,92.0,6674.0,223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15051,2021-05-19 00:00:00,Andhra Pradesh,1254291.0,9580.0,1475372.0,211501.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15052,2021-05-19 00:00:00,Arunachal Pradesh,19977.0,88.0,22462.0,2397.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15053,2021-05-19 00:00:00,Assam,290774.0,2344.0,340858.0,47740.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15054,2021-05-19 00:00:00,Bihar,595377.0,4039.0,664115.0,64699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [240]:
Covid_Testing = Covid_Testing.rename(columns={'State/Union Territory':'State'})

In [241]:
india_cases = india_cases.append(Covid_Testing,sort=False)

In [242]:
india_cases.head()

,Date,State,Cured,Deaths,Confirmed,Active,Total Individuals Vaccinated,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,AEFI,18-30 years (Age),30-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered,Total Vaccinatons,TotalSamples,Negative,Positive
15050,2021-05-19 00:00:00,Andaman and Nicobar Islands,6359.0,92.0,6674.0,223.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15051,2021-05-19 00:00:00,Andhra Pradesh,1254291.0,9580.0,1475372.0,211501.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15052,2021-05-19 00:00:00,Arunachal Pradesh,19977.0,88.0,22462.0,2397.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15053,2021-05-19 00:00:00,Assam,290774.0,2344.0,340858.0,47740.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15054,2021-05-19 00:00:00,Bihar,595377.0,4039.0,664115.0,64699.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [246]:
india_cases["Date"]=pd.to_datetime(india_cases["Date"])

In [247]:
india_cases = india_cases.sort_values(by="Date")

In [251]:
india_cases=india_cases.fillna(0)